In [1]:
import pandas as pd 
import numpy as np 

In [61]:
postings_df = pd.read_csv('indeed_data_scientist_roles.csv', sep='~')
postings_df

,position,rating,location,company_name,salary,posted_date,description
0,Data Scientist,NaN,+1 location Remote,CyberData Technologies,Full-time,Posted Posted 14 days ago,Apply data models to data sets. Utilizes simpl...
1,Data Scientist,NaN,"Remote in Wellesley Hills, MA 02481",Sun Life,"$84,600 - $126,900 a year",Posted Posted 14 days ago,"Utilize multiple sources of data, including st..."
2,Data Scientist,NaN,"New York, NY +1 location",Anheuser-Busch,"$106,650 - $195,600 a year",Posted Posted 7 days ago,"As a data scientist, you will work at the inte..."
3,Entry Level - Associate Data Scientist (Commer...,NaN,"New York, NY 10001 (Chelsea area)",IBM,"$95,000 - $123,000 a year",Posted Posted 1 day ago,"Working in an Agile, collaborative environment..."
4,Data Scientist,NaN,Remote,Red Hat Software,Full-time,Posted Posted 30+ days ago,"In this role, you will develop and modify soft..."
...,...,...,...,...,...,...,...
281,Data Scientist,NaN,"Springfield, VA 22152 +1 location",Absolute Business Solutions Corp,Estimated $143K - $181K a year,Posted Posted 18 days ago,Interprets results from multiple sources using...
282,Data Scientist,NaN,"Salt Lake City, UT 84112 (East Bench area)",University of Utah,"$75,000 - $130,000 a year",Posted Posted 30+ days ago,Oversees/conduct data science operations using...
283,Data Scientist,NaN,"New York, NY",Sriven System,Estimated $132K - $167K a year,Posted Posted 30+ days ago,The Data Engineer will play a key role in deve...
284,Biomedical Data Scientist,NaN,"White Plains, NY 10604",Retia Medical,Estimated $171K - $216K a year,Posted Posted 30+ days ago,Other responsibilities include maintaining Ret...


In [12]:
states_abbreviation = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY"
}


In [67]:
postings_df.position.value_counts()

Data Scientist                                                   111
Senior Data Scientist                                             13
Junior Data Scientist                                              4
Data Scientist II                                                  4
Data Scientist (Remote)                                            4
                                                                ... 
Data Scientist - CX Analytics                                      1
Data Associate Scientist                                           1
Associate Data Scientist - Online Business Analytics (Remote)      1
Early Career Data Scientist (Federal)                              1
Data Scientist - Club Analytics                                    1
Name: position, Length: 130, dtype: int64

In [62]:
# map each location to the state 
# compare the location of job postings to locations of big tech - predict the hidden market? 


states = postings_df.location.apply(lambda location_descr: [state for state, abbrev in states_abbreviation.items() if (abbrev in location_descr) or (state in location_descr)])
states = states.apply(lambda row: ' '.join(row))
postings_df['state'] = states

In [56]:
postings_df['state'].value_counts()

California        33
                  32
New York          28
Florida           14
Washington        13
Texas             12
Illinois          11
Pennsylvania       7
Massachusetts      6
North Carolina     5
Georgia            4
New Jersey         4
New Mexico         3
Virginia           3
Colorado           3
Maryland           3
Ohio               2
Tennessee          2
Arizona            2
Wisconsin          2
Nevada             2
Delaware           2
Arkansas           1
Kentucky           1
Maine              1
Iowa               1
Minnesota          1
Michigan           1
Missouri           1
Louisiana          1
Connecticut        1
Name: state, dtype: int64

In [63]:
import folium
from folium import plugins
import pandas as pd
from geopy.geocoders import Nominatim

# Create a dataframe from the counts
df = pd.DataFrame({'state':['California', 'New York', 'Illinois','Washington','Texas','Massachusetts','Ohio','Pennsylvania','Florida','Iowa','New Mexico','Arizona','Maryland','Wisconsin','Louisiana','Virginia'],
                  'count':[11,9,5,5,5,3,2,2,1,1,1,1,1,1,1,1]})

# Use geopy library to get the latitude and longitude of the states
geolocator = Nominatim(user_agent="myGeocoder")
df['location'] = df['state'].apply(geolocator.geocode)
df['latitude'] = df['location'].apply(lambda loc: loc.latitude)
df['longitude'] = df['location'].apply(lambda loc: loc.longitude)

# Create a map centered on the North America
m = folium.Map(location=[39, -98], zoom_start=4)

# Add the heat map
m.add_child(plugins.HeatMap(data=df[['latitude', 'longitude', 'count']].values.tolist(),
                            columns=['latitude', 'longitude', 'count'],
                            name='count'))

# Add a layer control to switch between the heat map and the states
folium.LayerControl().add_to(m)

# Display the map


In [64]:
m

In [65]:
m.save('map.html')


In [78]:
import re

# The regular expression pattern
pattern = r'\$\d{1,3}(,\d{3})*\s-\s\$\d{1,3}(,\d{3})*\s(a year)?'

postings_df['salary']#.apply(lambda salary_expr: re.findall(pattern, salary_expr))


0                           Full-time
1           $84,600 - $126,900 a year
2          $106,650 - $195,600 a year
3           $95,000 - $123,000 a year
4                           Full-time
                    ...              
281    Estimated $143K - $181K a year
282         $75,000 - $130,000 a year
283    Estimated $132K - $167K a year
284    Estimated $171K - $216K a year
285                         Full-time
Name: salary, Length: 286, dtype: object